# Scikit-learn - Aprendizaje No Supervisado

Este notebook cubre los algoritmos de aprendizaje no supervisado en scikit-learn: clustering y reducción de dimensionalidad.

## Importar Módulos


In [ ]:
from sklearn import datasets
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.decomposition import PCA, TruncatedSVD, NMF
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt


## Clustering


In [ ]:
# Cargar datos
X, y = datasets.make_blobs(n_samples=300, centers=4, random_state=42)

# K-Means
kmeans = KMeans(n_clusters=4, random_state=42, n_init=10)
kmeans_labels = kmeans.fit_predict(X)
print(f"K-Means - Número de clusters: {len(np.unique(kmeans_labels))}")
print(f"Centroides: {kmeans.cluster_centers_.shape}")
print(f"Inercia (suma de distancias al cuadrado): {kmeans.inertia_:.2f}")

# DBSCAN (basado en densidad)
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_labels = dbscan.fit_predict(X)
n_clusters_dbscan = len(set(dbscan_labels)) - (1 if -1 in dbscan_labels else 0)
n_noise = list(dbscan_labels).count(-1)
print(f"\nDBSCAN - Número de clusters: {n_clusters_dbscan}")
print(f"Puntos ruidosos: {n_noise}")

# Clustering Jerárquico Aglomerativo
agg = AgglomerativeClustering(n_clusters=4)
agg_labels = agg.fit_predict(X)
print(f"\nAgglomerativeClustering - Número de clusters: {len(np.unique(agg_labels))}")


## Reducción de Dimensionalidad


In [ ]:
# Cargar datos de alta dimensionalidad
X, y = datasets.load_iris(return_X_y=True)
X_scaled = StandardScaler().fit_transform(X)

# PCA (Análisis de Componentes Principales)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_scaled)
print("PCA:")
print(f"Varianza explicada por componente: {pca.explained_variance_ratio_}")
print(f"Varianza total explicada: {pca.explained_variance_ratio_.sum():.4f}")
print(f"Shape original: {X.shape}, Shape reducido: {X_pca.shape}")

# TruncatedSVD (SVD truncado, útil para matrices sparse)
svd = TruncatedSVD(n_components=2, random_state=42)
X_svd = svd.fit_transform(X_scaled)
print(f"\nTruncatedSVD:")
print(f"Varianza explicada: {svd.explained_variance_ratio_.sum():.4f}")

# NMF (Non-negative Matrix Factorization)
# Requiere datos no negativos
X_positive = X_scaled - X_scaled.min() + 0.1  # Hacer valores positivos
nmf = NMF(n_components=2, random_state=42)
X_nmf = nmf.fit_transform(X_positive)
print(f"\nNMF:")
print(f"Shape reducido: {X_nmf.shape}")


## t-SNE (t-Distributed Stochastic Neighbor Embedding)

In [ ]:
# t-SNE: útil para visualización (no para reducción general)
# Nota: t-SNE es computacionalmente costoso para datasets grandes
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_scaled)
print("t-SNE:")
print(f"Shape original: {X.shape}, Shape reducido: {X_tsne.shape}")
print("Nota: t-SNE es principalmente para visualización, no para reducción de características")


## Selección del Número de Componentes (PCA)


In [ ]:
# PCA con todos los componentes para ver varianza explicada
pca_full = PCA()
pca_full.fit(X_scaled)

# Calcular varianza acumulada
cumulative_variance = np.cumsum(pca_full.explained_variance_ratio_)
print("Varianza explicada acumulada por número de componentes:")
for i, var in enumerate(cumulative_variance, 1):
    print(f"  {i} componentes: {var:.4f}")

# Seleccionar número de componentes que explican el 95% de la varianza
n_components_95 = np.argmax(cumulative_variance >= 0.95) + 1
print(f"\nNúmero de componentes para 95% de varianza: {n_components_95}")

# Aplicar PCA con número óptimo
pca_optimal = PCA(n_components=n_components_95)
X_pca_optimal = pca_optimal.fit_transform(X_scaled)
print(f"Shape reducido: {X_pca_optimal.shape}")
